In [65]:
import pandas as pd
import numpy as np
import random
import string

pd.set_option("display.max_columns", 200)
pd.set_option("display.max_rows", 1500)
pd.set_option("display.max_colwidth", None)

## Load the data

In [2]:
balances_df = pd.read_csv("balances.csv", sep=";")
collateral_df = pd.read_csv("collateral.csv", sep=";")
customers_df = pd.read_csv("customers.csv", sep=";")

In [3]:
balances_df.head()

,reporting_date,customer_id,total_cust_exposure,overdraft,consumer_loan,credit_card,customer_rating,account_id,open_date,maturity_date,original_principal,current_principal,interest_rate,prepayment_status
0,2022-03-31,0x0ADCBD160AD1D6F746,114368,0,0,0,4,0x31C39D093457158DD67CE37D9414A489,2021-08-11,2036-08-11,118200,114368,2.55,no
1,2022-04-29,0x0ADCBD160AD1D6F746,113826,0,0,0,5,0x31C39D093457158DD67CE37D9414A489,2021-08-11,2036-08-11,118200,113826,2.55,no
2,2022-05-31,0x0ADCBD160AD1D6F746,113275,0,0,0,4,0x31C39D093457158DD67CE37D9414A489,2021-08-11,2036-08-11,118200,113275,2.55,no
3,2022-02-28,0x6C3B750BA31321613B,110088,0,0,0,4+,0xF9E7CC9831573E94280AA11FD5C7A883,2021-08-12,2051-08-12,111384,110088,2.55,no
4,2022-06-30,0x6C3B750BA31321613B,109241,0,0,0,4+,0xF9E7CC9831573E94280AA11FD5C7A883,2021-08-12,2051-08-12,111384,109241,2.55,no


In [4]:
collateral_df.head()

,account_id,collateral_id,market_value,collateral_type,building_type,mortgage_utilisation,town,area
0,0x41702C36588DDB37F2DD861C5DB40111,0x82164E87B8688CBEC83A21BD4D,136800,M100,1.0,1.0,Plovdiv,84.276
1,0x05354063C1E883849C745C1F1B0418D9,0xA7B4F5E66A9AA942428F810CF8,397800,M101,1.0,1.0,Plovdiv,344.172
2,0x9A6BED3374ABA6453F5BD729DF4E3E01,0x845E56B7B510F20CA2AB7FAB21,95237,M100,2.0,1.0,Sofia,50.448
3,0x480CA3D2B69E387AEE0040B33C0FA21C,0xA069E952426A04A9529D9C011A,98574,M100,1.0,1.0,Plovdiv,116.460
4,0xC6CB9AE910C26A4C142CB6600E8599D6,0xC76BF79BB907B0F68DF52F59FE,187560,M100,1.0,1.0,Plovdiv,161.088


In [5]:
customers_df.head()

,customer_id,age,maritial_status,town,empl_type,profession,tot_mnth_income,gross_salary
0,0x966001FD3EF723FC5F,Q251,U67,Sofia,I508,2,5138,6000
1,0xBA04902FFAA73BF01A,Z829,K16,Smolyan,Q237,11,3551,119
2,0xC28B6C6C55416E68A9,Z829,A49,Plovdiv,R412,6,1384,119
3,0x07C6BA3D65466D8A6A,J454,U67,Sofia,I508,41,1652,1128
4,0xF49D6CCED11944AB48,G219,U67,Sofia,I508,10,9655,11225


### Shapes

In [6]:
# WILL JOIN BASED ON JOINS BETWEEN balances AND THESE TWO DFs
set(customers_df["customer_id"]) & set(collateral_df["account_id"])

set()

In [7]:
len(set(customers_df["customer_id"]) & set(balances_df["customer_id"]))

34515

In [8]:
len(set(collateral_df["account_id"]) & set(balances_df["account_id"]))

39520

In [9]:
# balances_df["maturity_date"] = pd.to_datetime(balances_df["maturity_date"])
# balances_df["open_date"] = pd.to_datetime(balances_df["open_date"])

# l_period = (balances_df.loc[:, "maturity_date"] - balances_df.loc[:, "open_date"]) / 365.25
# l_period = l_period.apply(lambda x: int(round(x.days, 0)))
# # l_period[0].days
# balances_df["l_period"] = l_period
    
# loan_period_y = 25    
# balances_df["exp_monthly_payments"] = balances_df.loc[:, "original_principal"]*(balances_df.loc[:, "interest_rate"]/100/12*(1+balances_df.loc[:, "interest_rate"]/100/12)**(12*balances_df.loc[:, "l_period"]) / ((1+balances_df.loc[:, "interest_rate"]/100/12)**(12*balances_df.loc[:, "l_period"]) - 1))
# grouped_data["exp_monthly_payments"] = grouped_data.loc[:, "original_principal"]*(grouped_data.loc[:, "interest_rate"]/100/12*(1+grouped_data.loc[:, "interest_rate"]/100/12)**(12*balances_df.loc[:, "l_period"]) / ((1+balances_df.loc[:, "interest_rate"]/100/12)**(12*balances_df.loc[:, "l_period"]) - 1))


In [10]:
# grouped_data["exp_monthly_payments"] = grouped_data.loc[:, "original_principal"]*(grouped_data.loc[:, "interest_rate"]/100/12*(1+grouped_data.loc[:, "interest_rate"]/100/12)**(12*grouped_data.loc[:, "l_period"]) / ((1+grouped_data.loc[:, "interest_rate"]/100/12)**(12*grouped_data.loc[:, "l_period"]) - 1))
# grouped_data["exp_monthly_payments"] = grouped_data.loc[:, "original_principal"]*(grouped_data.loc[:, "interest_rate"]/100/12*(1+grouped_data.loc[:, "interest_rate"]/100/12)**(12*grouped_data.loc[:, "l_period"]) / ((1+grouped_data.loc[:, "interest_rate"]/100/12)**(12*grouped_data.loc[:, "l_period"]) - 1))


In [11]:
balances_df.head()

,reporting_date,customer_id,total_cust_exposure,overdraft,consumer_loan,credit_card,customer_rating,account_id,open_date,maturity_date,original_principal,current_principal,interest_rate,prepayment_status
0,2022-03-31,0x0ADCBD160AD1D6F746,114368,0,0,0,4,0x31C39D093457158DD67CE37D9414A489,2021-08-11,2036-08-11,118200,114368,2.55,no
1,2022-04-29,0x0ADCBD160AD1D6F746,113826,0,0,0,5,0x31C39D093457158DD67CE37D9414A489,2021-08-11,2036-08-11,118200,113826,2.55,no
2,2022-05-31,0x0ADCBD160AD1D6F746,113275,0,0,0,4,0x31C39D093457158DD67CE37D9414A489,2021-08-11,2036-08-11,118200,113275,2.55,no
3,2022-02-28,0x6C3B750BA31321613B,110088,0,0,0,4+,0xF9E7CC9831573E94280AA11FD5C7A883,2021-08-12,2051-08-12,111384,110088,2.55,no
4,2022-06-30,0x6C3B750BA31321613B,109241,0,0,0,4+,0xF9E7CC9831573E94280AA11FD5C7A883,2021-08-12,2051-08-12,111384,109241,2.55,no


In [12]:
balances_df.loc[balances_df["customer_id"] == "0x000100F424893C746F"]

,reporting_date,customer_id,total_cust_exposure,overdraft,consumer_loan,credit_card,customer_rating,account_id,open_date,maturity_date,original_principal,current_principal,interest_rate,prepayment_status
45791,2022-02-28,0x000100F424893C746F,56789,0,0,0,5+,0xF7CDAD654B6CE06334F9FE655212AAD1,2017-06-29,2042-06-29,64800,56789,3.7,no
45792,2022-06-30,0x000100F424893C746F,56155,0,0,0,3-,0xF7CDAD654B6CE06334F9FE655212AAD1,2017-06-29,2042-06-29,64800,56155,3.7,no
62060,2022-01-31,0x000100F424893C746F,56941,0,0,0,3+,0xF7CDAD654B6CE06334F9FE655212AAD1,2017-06-29,2042-06-29,64800,56941,3.7,no
62725,2022-03-31,0x000100F424893C746F,56620,0,0,0,3-,0xF7CDAD654B6CE06334F9FE655212AAD1,2017-06-29,2042-06-29,64800,56620,3.7,no
62726,2022-04-29,0x000100F424893C746F,56467,0,0,0,3-,0xF7CDAD654B6CE06334F9FE655212AAD1,2017-06-29,2042-06-29,64800,56467,3.7,no
62727,2022-05-31,0x000100F424893C746F,56308,0,0,0,3-,0xF7CDAD654B6CE06334F9FE655212AAD1,2017-06-29,2042-06-29,64800,56308,3.7,no
292730,2021-01-29,0x000100F424893C746F,59085,0,0,0,4+,0xF7CDAD654B6CE06334F9FE655212AAD1,2017-06-29,2042-06-29,64800,58765,3.7,no
292731,2021-11-30,0x000100F424893C746F,57249,0,0,0,3+,0xF7CDAD654B6CE06334F9FE655212AAD1,2017-06-29,2042-06-29,64800,57249,3.7,no
325941,2021-02-26,0x000100F424893C746F,59029,0,0,0,3-,0xF7CDAD654B6CE06334F9FE655212AAD1,2017-06-29,2042-06-29,64800,58765,3.7,no
329302,2021-04-29,0x000100F424893C746F,58308,0,0,0,3+,0xF7CDAD654B6CE06334F9FE655212AAD1,2017-06-29,2042-06-29,64800,58308,3.7,no


In [13]:
def add_paym_details(group):
    
    """
    Creates Series of expected payment amounts per period; expected principle payment; expected interest payment; actual_principle_deduction; difference between deducted vs actual principle payment.
    
    Expects grouped_data to be a grouped dataFrame which contains info for single client"""
    
    
    label, grouped_data = group
    
    grouped_data.sort_values(by="reporting_date", inplace=True)
    grouped_data.reset_index(inplace=True)
    
    grouped_data["maturity_date"] = pd.to_datetime(grouped_data["maturity_date"])
    grouped_data["open_date"] = pd.to_datetime(grouped_data["open_date"])
    
    
    
#     loan_period = grouped_data.loc[0, "maturity_date"] - grouped_data.loc[0, "open_date"]
#     loan_period_y = int(round(loan_period.days / 365.25, 0))

    l_period = (grouped_data.loc[:, "maturity_date"] - grouped_data.loc[:, "open_date"]) / 365.25
    l_period = l_period.apply(lambda x: int(round(x.days, 0)))

        
#     P = grouped_data.loc[0, "original_principal"]
#     n = years*12
    
    grouped_data["l_period"] = l_period
    grouped_data["exp_monthly_payments"] = grouped_data.loc[:, "original_principal"]*(grouped_data.loc[:, "interest_rate"]/100/12*(1+grouped_data.loc[:, "interest_rate"]/100/12)**(12*grouped_data.loc[:, "l_period"]) / ((1+grouped_data.loc[:, "interest_rate"]/100/12)**(12*grouped_data.loc[:, "l_period"]) - 1))
    grouped_data["interest_payment"] = grouped_data["current_principal"]*(grouped_data["interest_rate"]/100/12)
    grouped_data["principal_payment"] = grouped_data["exp_monthly_payments"] - grouped_data["interest_payment"]
    
    for row in range(len(grouped_data)):
        if row == 0:
            grouped_data.loc[row, "deducted_principal"] = grouped_data.loc[row, "original_principal"] - grouped_data.loc[row, "current_principal"]
            grouped_data.loc[row, "deducted_vs_principal_payment"] = grouped_data.loc[row, "deducted_principal"] - grouped_data.loc[row, "principal_payment"]
            grouped_data.loc[row, "deducted_vs_principal_payment_ratio"] = (grouped_data.loc[row, "deducted_principal"] / grouped_data.loc[row, "principal_payment"]) * 100
            
        else:
            grouped_data.loc[row, "deducted_principal"] = (grouped_data.loc[row-1, "current_principal"] - grouped_data.loc[row, "current_principal"])
            grouped_data.loc[row, "deducted_vs_principal_payment"] = grouped_data.loc[row, "deducted_principal"] - grouped_data.loc[row, "principal_payment"]
            grouped_data.loc[row, "deducted_vs_principal_payment_ratio"] = (grouped_data.loc[row, "deducted_principal"] / grouped_data.loc[row, "principal_payment"]) * 100    
    
    
    grouped_data["principal_left_pctg"] = round(grouped_data["current_principal"] / grouped_data["original_principal"] * 100)
    
    return grouped_data

## Calculating Loan Payments

In [14]:
balances_grouped = balances_df.groupby(balances_df["customer_id"])

In [15]:
balances_updated = pd.DataFrame()

for i in balances_grouped:
    updated_df = add_paym_details(i)
    balances_updated = pd.concat([balances_updated, updated_df])

In [16]:
balances_updated.to_csv("balances_updated_v4.csv")

In [17]:
balances_updated[balances_updated.isnull().any(axis=1)]

,index,reporting_date,customer_id,total_cust_exposure,overdraft,consumer_loan,credit_card,customer_rating,account_id,open_date,maturity_date,original_principal,current_principal,interest_rate,prepayment_status,l_period,exp_monthly_payments,interest_payment,principal_payment,deducted_principal,deducted_vs_principal_payment,deducted_vs_principal_payment_ratio,principal_left_pctg
17,568996,2020-05-29,0x0005F1CA4970B6DBEA,141001,0,0,0,4,0x8433D2AFE12E765CA524977790A6D485,2017-07-03,2053-01-05,147000,140237,0.0,refinanced other banks,35,NaN,0.0,NaN,208.0,NaN,NaN,95.0
18,568997,2020-06-30,0x0005F1CA4970B6DBEA,140972,0,0,0,3,0x8433D2AFE12E765CA524977790A6D485,2017-07-03,2053-01-05,147000,140237,0.0,refinanced other banks,35,NaN,0.0,NaN,0.0,NaN,NaN,95.0
19,699800,2020-07-31,0x0005F1CA4970B6DBEA,140968,0,0,0,3+,0x8433D2AFE12E765CA524977790A6D485,2017-07-03,2053-01-05,147000,140237,0.0,refinanced other banks,35,NaN,0.0,NaN,0.0,NaN,NaN,95.0
20,699739,2020-08-31,0x0005F1CA4970B6DBEA,140951,0,0,0,4-,0x8433D2AFE12E765CA524977790A6D485,2017-07-03,2053-01-05,147000,140237,0.0,refinanced other banks,35,NaN,0.0,NaN,0.0,NaN,NaN,95.0
21,855268,2020-09-30,0x0005F1CA4970B6DBEA,140983,0,0,0,4+,0x8433D2AFE12E765CA524977790A6D485,2017-07-03,2053-01-05,147000,140237,0.0,refinanced other banks,35,NaN,0.0,NaN,0.0,NaN,NaN,95.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20,875389,2020-08-31,0xFFF7DC050B8B833EE8,35320,0,0,0,8,0x2B6893D03653A868FC99E1C20D96563B,2005-12-09,2024-06-20,140820,33218,0.0,no,18,NaN,0.0,NaN,0.0,NaN,NaN,24.0
21,712394,2020-09-30,0xFFF7DC050B8B833EE8,34404,0,0,0,7,0x2B6893D03653A868FC99E1C20D96563B,2005-12-09,2024-06-20,140820,33218,0.0,no,18,NaN,0.0,NaN,0.0,NaN,NaN,24.0
22,914491,2020-10-30,0xFFF7DC050B8B833EE8,33708,0,0,0,7,0x2B6893D03653A868FC99E1C20D96563B,2005-12-09,2024-06-20,140820,33218,0.0,no,18,NaN,0.0,NaN,0.0,NaN,NaN,24.0
26,247914,2021-02-26,0xFFF7DC050B8B833EE8,33645,0,0,0,6+,0x2B6893D03653A868FC99E1C20D96563B,2005-12-09,2024-09-20,140820,31204,0.0,no,18,NaN,0.0,NaN,0.0,NaN,NaN,22.0


In [148]:
balances_updated["deducted_vs_principal_payment_ratio"]

0      1665.332908
1        90.533639
2        90.953476
3       104.092900
4        90.408829
          ...     
27       79.773822
28       71.062402
29       73.947546
30       71.303291
0     14930.284332
Name: deducted_vs_principal_payment_ratio, Length: 1230563, dtype: float64

In [19]:
balances_updated.fillna(0, inplace=True)

In [20]:
balances_updated.shape

(1230563, 23)

In [21]:
balances_updated.head(100)

,index,reporting_date,customer_id,total_cust_exposure,overdraft,consumer_loan,credit_card,customer_rating,account_id,open_date,maturity_date,original_principal,current_principal,interest_rate,prepayment_status,l_period,exp_monthly_payments,interest_payment,principal_payment,deducted_principal,deducted_vs_principal_payment,deducted_vs_principal_payment_ratio,principal_left_pctg
0,1161746,2018-12-31,0x000100F424893C746F,63337,0,0,0,3+,0xF7CDAD654B6CE06334F9FE655212AAD1,2017-06-29,2042-06-29,64800,62342,3.70,no,24,339.819283,192.221167,147.598116,2458.0,2310.401884,1665.332908,96.0
1,1163788,2019-01-31,0x000100F424893C746F,63355,0,0,0,3+,0xF7CDAD654B6CE06334F9FE655212AAD1,2017-06-29,2042-06-29,64800,62208,3.70,no,24,339.819283,191.808000,148.011283,134.0,-14.011283,90.533639,96.0
2,873261,2019-02-28,0x000100F424893C746F,63112,0,0,0,4+,0xF7CDAD654B6CE06334F9FE655212AAD1,2017-06-29,2042-06-29,64800,62073,3.70,no,24,339.819283,191.391750,148.427533,135.0,-13.427533,90.953476,96.0
3,1194608,2019-03-29,0x000100F424893C746F,62851,0,0,0,3,0xF7CDAD654B6CE06334F9FE655212AAD1,2017-06-29,2042-06-29,64800,61918,3.70,no,24,339.819283,190.913833,148.905450,155.0,6.094550,104.092900,96.0
4,1093031,2019-04-30,0x000100F424893C746F,62607,0,0,0,3+,0xF7CDAD654B6CE06334F9FE655212AAD1,2017-06-29,2042-06-29,64800,61783,3.70,no,24,339.819283,190.497583,149.321700,135.0,-14.321700,90.408829,95.0
5,1167389,2019-05-31,0x000100F424893C746F,62355,0,0,0,3,0xF7CDAD654B6CE06334F9FE655212AAD1,2017-06-29,2042-06-29,64800,61640,3.70,no,24,339.819283,190.056667,149.762616,143.0,-6.762616,95.484443,95.0
6,915280,2019-06-28,0x000100F424893C746F,62551,0,0,0,3,0xF7CDAD654B6CE06334F9FE655212AAD1,2017-06-29,2042-06-29,64800,61503,3.70,no,24,339.819283,189.634250,150.185033,137.0,-13.185033,91.220808,95.0
7,982182,2019-07-31,0x000100F424893C746F,62302,0,0,0,3,0xF7CDAD654B6CE06334F9FE655212AAD1,2017-06-29,2042-06-29,64800,61360,3.70,no,24,339.819283,189.193333,150.625950,143.0,-7.625950,94.937161,95.0
8,873262,2019-08-30,0x000100F424893C746F,62056,0,0,0,3+,0xF7CDAD654B6CE06334F9FE655212AAD1,2017-06-29,2042-06-29,64800,61222,3.70,no,24,339.819283,188.767833,151.051450,138.0,-13.051450,91.359600,94.0
9,1083187,2019-09-30,0x000100F424893C746F,61950,0,0,0,4,0xF7CDAD654B6CE06334F9FE655212AAD1,2017-06-29,2042-06-29,64800,61084,3.70,no,24,339.819283,188.342333,151.476950,138.0,-13.476950,91.102970,94.0


In [161]:
balances_updated.loc[balances_updated["customer_id"] == "0x000224F02B4B39DDFE"]

,index,reporting_date,customer_id,total_cust_exposure,overdraft,consumer_loan,credit_card,customer_rating,account_id,open_date,maturity_date,original_principal,current_principal,interest_rate,prepayment_status,l_period,exp_monthly_payments,interest_payment,principal_payment,deducted_principal,deducted_vs_principal_payment,deducted_vs_principal_payment_ratio,principal_left_pctg
0,1228553,2018-12-31,0x000224F02B4B39DDFE,71273,0,0,0,5+,0x5568E6E4AEB2DF832D6444950125EECB,2016-08-16,2042-08-16,74880,71273,3.9,refinanced other banks,25,391.121433,231.63725,159.484183,3607.0,3447.515817,2261.666292,95.0
1,1056541,2019-01-31,0x000224F02B4B39DDFE,71372,0,0,0,5+,0x5568E6E4AEB2DF832D6444950125EECB,2016-08-16,2042-08-16,74880,71126,3.9,refinanced other banks,25,391.121433,231.15950,159.961933,147.0,-12.961933,91.896864,95.0
2,1211827,2019-02-28,0x000224F02B4B39DDFE,71929,0,0,0,4+,0x5568E6E4AEB2DF832D6444950125EECB,2016-08-16,2042-08-16,74880,70978,3.9,refinanced other banks,25,391.121433,230.67850,160.442933,148.0,-12.442933,92.244636,95.0
3,1000865,2019-03-29,0x000224F02B4B39DDFE,70806,0,0,0,4+,0x5568E6E4AEB2DF832D6444950125EECB,2016-08-16,2042-08-16,74880,70806,3.9,refinanced other banks,25,391.121433,230.11950,161.001933,172.0,10.998067,106.831016,95.0
4,1056619,2019-04-30,0x000224F02B4B39DDFE,71556,0,0,0,4,0x5568E6E4AEB2DF832D6444950125EECB,2016-08-16,2042-08-16,74880,70657,3.9,refinanced other banks,25,391.121433,229.63525,161.486183,149.0,-12.486183,92.267956,94.0
5,1043623,2019-05-31,0x000224F02B4B39DDFE,71672,0,0,0,4,0x5568E6E4AEB2DF832D6444950125EECB,2016-08-16,2042-08-16,74880,70500,3.9,refinanced other banks,25,391.121433,229.12500,161.996433,157.0,-4.996433,96.915714,94.0
6,1077553,2019-06-28,0x000224F02B4B39DDFE,72618,0,0,0,4,0x5568E6E4AEB2DF832D6444950125EECB,2016-08-16,2042-08-16,74880,70350,3.9,refinanced other banks,25,391.121433,228.63750,162.483933,150.0,-12.483933,92.316820,94.0
7,953281,2019-07-31,0x000224F02B4B39DDFE,72448,0,0,0,4,0x5568E6E4AEB2DF832D6444950125EECB,2016-08-16,2042-08-16,74880,70191,3.9,refinanced other banks,25,391.121433,228.12075,163.000683,159.0,-4.000683,97.545604,94.0
8,986928,2019-08-30,0x000224F02B4B39DDFE,72161,0,0,0,4-,0x5568E6E4AEB2DF832D6444950125EECB,2016-08-16,2042-08-16,74880,70040,3.9,refinanced other banks,25,391.121433,227.63000,163.491433,151.0,-12.491433,92.359580,94.0
9,1173420,2019-09-30,0x000224F02B4B39DDFE,72710,0,0,0,4-,0x5568E6E4AEB2DF832D6444950125EECB,2016-08-16,2042-08-16,74880,69888,3.9,refinanced other banks,25,391.121433,227.13600,163.985433,152.0,-11.985433,92.691160,93.0


In [ ]:
balances_updated["prepayment_status"].unique()

In [ ]:
balances_updated.loc[balances_updated["prepayment_status"] == "own"]["customer_id"].unique()

In [ ]:
balances_updated.loc[balances_updated["customer_id"] == "0xFFFAF7DD151CBA684C"]

In [ ]:
# balances_updated = pd.read_csv("balances_updated_v1.csv")
# balances_updated.drop("Unnamed: 0", axis=1, inplace=True)

In [ ]:
# balances_updated.info()

In [ ]:
# balances_updated[balances_updated.isnull().any(axis=1)]

In [ ]:
# balances_updated.dropna(inplace=True)

In [23]:
balances_updated.to_csv("balances_updated_v4.csv")

## Feature Engineering

### Create features

In [ ]:
# balances_updated = pd.read_csv("balances_updated_v4.csv")
# balances_updated.drop("Unnamed: 0", axis=1, inplace=True)

In [ ]:
# balances_updated = pd.read_csv("balances_updated_v2.csv")
# balances_updated.drop("Unnamed: 0", axis=1, inplace=True)

Balances will be grouped on:

- reporting_date = count = "count_reporting_date"
- customer_id = unique = "customer_id"
- total_cust_exposure = ????????? maybe average_cust_exposure
- overdraft = max - min = diff_overdraft
- overdraft_0s = count how many zeros in overdraft column
- overdraft_1s = count how many ones in overdraft column
- overdraft_2s = count how many twos in overdraft column
- consumer_loan = max - min = diff_consumer_loan



- customer_rating??????  - standard metric regarding the ability of the client to perform payments
- customer_rating?????? = max = max_customer_rating
- customer_rating = min = min_customer_rating
- customer_rating = last = last_customer_rating
- account_id = unique = account_id
- mount of full payment - count after how many months the credit was fully prepaid

In [ ]:
l = [1, 2, 3, 8, 8]
l.count(8)

In [ ]:
ser = pd.Series([1,2,3,3,4,5,6,4])
len(np.where(ser>=5)[0])

In [86]:
["5","4","5","4","3","2","3","4","5"].mode()

AttributeError: 'list' object has no attribute 'mode'

In [99]:
balances_updated["customer_rating"].value_counts().idxmax()

'4+'

In [100]:
def count0(x):
    return list(x).count(0)

def count1(x):
    return list(x).count(1)

def count2plus(x):
    return len(np.where(x>=2)[0])

def modeX(x):
    return x.value_counts().idxmax()

def max_minus_min(x):
    return x.max() - x.min()



In [101]:
balances_updated.columns

Index(['index', 'reporting_date', 'customer_id', 'total_cust_exposure',
       'overdraft', 'consumer_loan', 'credit_card', 'customer_rating',
       'account_id', 'open_date', 'maturity_date', 'original_principal',
       'current_principal', 'interest_rate', 'prepayment_status', 'l_period',
       'exp_monthly_payments', 'interest_payment', 'principal_payment',
       'deducted_principal', 'deducted_vs_principal_payment',
       'deducted_vs_principal_payment_ratio', 'principal_left_pctg'],
      dtype='object')

In [162]:
balances_updated.loc[balances_updated["customer_id"] == "0x0004EAC7528D4869C1"]

,index,reporting_date,customer_id,total_cust_exposure,overdraft,consumer_loan,credit_card,customer_rating,account_id,open_date,maturity_date,original_principal,current_principal,interest_rate,prepayment_status,l_period,exp_monthly_payments,interest_payment,principal_payment,deducted_principal,deducted_vs_principal_payment,deducted_vs_principal_payment_ratio,principal_left_pctg
0,411624,2021-08-31,0x0004EAC7528D4869C1,96171,0,0,0,4-,0x1DE89AAB80B0FE12FBA0E35F21DDA118,2021-07-07,2051-07-07,118320,96171,2.49,no,29,477.741699,199.554825,278.186874,22149.0,21870.813126,7961.914117,81.0
1,494017,2021-09-30,0x0004EAC7528D4869C1,95996,0,0,0,4-,0x1DE89AAB80B0FE12FBA0E35F21DDA118,2021-07-07,2051-07-07,118320,95996,2.49,no,29,477.741699,199.191700,278.549999,175.0,-103.549999,62.825346,81.0
2,323749,2021-10-29,0x0004EAC7528D4869C1,95813,0,0,0,4-,0x1DE89AAB80B0FE12FBA0E35F21DDA118,2021-07-07,2051-07-07,118320,95813,2.49,no,29,477.741699,198.811975,278.929724,183.0,-95.929724,65.607923,81.0
3,513469,2021-11-30,0x0004EAC7528D4869C1,95636,0,0,0,4-,0x1DE89AAB80B0FE12FBA0E35F21DDA118,2021-07-07,2051-07-07,118320,95636,2.49,no,29,477.741699,198.444700,279.296999,177.0,-102.296999,63.373398,81.0
4,178327,2021-12-31,0x0004EAC7528D4869C1,95453,0,0,0,4-,0x1DE89AAB80B0FE12FBA0E35F21DDA118,2021-07-07,2051-07-07,118320,95453,2.49,no,29,477.741699,198.064975,279.676724,183.0,-96.676724,65.432689,81.0
5,15368,2022-01-31,0x0004EAC7528D4869C1,95275,0,0,0,4-,0x1DE89AAB80B0FE12FBA0E35F21DDA118,2021-07-07,2051-07-07,118320,95275,2.49,no,29,477.741699,197.695625,280.046074,178.0,-102.046074,63.560970,81.0
6,585709,2022-01-31,0x0004EAC7528D4869C1,95275,0,0,0,4-,0x1DE89AAB80B0FE12FBA0E35F21DDA118,2021-07-07,2051-07-07,118320,95275,2.49,no,29,477.741699,197.695625,280.046074,0.0,-280.046074,0.000000,81.0
7,65408,2022-02-28,0x0004EAC7528D4869C1,116897,0,0,0,3,0x1DE89AAB80B0FE12FBA0E35F21DDA118,2021-07-07,2051-07-07,118320,116897,2.49,no,29,477.741699,242.561275,235.180424,-21622.0,-21857.180424,-9193.792418,99.0
8,29245,2022-03-31,0x0004EAC7528D4869C1,116644,0,0,0,4,0x1DE89AAB80B0FE12FBA0E35F21DDA118,2021-07-07,2051-07-07,118320,116644,2.49,no,29,477.741699,242.036300,235.705399,253.0,17.294601,107.337380,99.0
9,29246,2022-04-29,0x0004EAC7528D4869C1,116424,0,0,0,4,0x1DE89AAB80B0FE12FBA0E35F21DDA118,2021-07-07,2051-07-07,118320,116424,2.49,no,29,477.741699,241.579800,236.161899,220.0,-16.161899,93.156432,98.0


In [102]:
balances_updated.loc[balances_updated["customer_id"] == "0xE535FBD1AB5E3C4C21"]

,index,reporting_date,customer_id,total_cust_exposure,overdraft,consumer_loan,credit_card,customer_rating,account_id,open_date,maturity_date,original_principal,current_principal,interest_rate,prepayment_status,l_period,exp_monthly_payments,interest_payment,principal_payment,deducted_principal,deducted_vs_principal_payment,deducted_vs_principal_payment_ratio,principal_left_pctg
0,1182498,2018-12-31,0xE535FBD1AB5E3C4C21,81095,0,0,0,3+,0xE92ADDF60D9801351C338B2608632B61,2017-10-18,2052-10-18,82800,80728,0.75,no,35,224.209839,50.455000,173.754839,2072.0,1898.245161,1192.484776,97.0
1,1058229,2019-01-31,0xE535FBD1AB5E3C4C21,79055,0,0,0,3+,0xE92ADDF60D9801351C338B2608632B61,2017-10-18,2052-03-01,82800,79055,0.75,no,34,229.977744,49.409375,180.568369,1673.0,1492.431631,926.518863,95.0
2,1036579,2019-02-28,0xE535FBD1AB5E3C4C21,78881,0,0,0,3+,0xE92ADDF60D9801351C338B2608632B61,2017-10-18,2052-03-01,82800,78881,0.75,no,34,229.977744,49.300625,180.677119,174.0,-6.677119,96.304391,95.0
3,1048685,2019-03-29,0xE535FBD1AB5E3C4C21,77142,0,0,0,3+,0xE92ADDF60D9801351C338B2608632B61,2017-10-18,2051-06-01,82800,77142,0.75,no,33,236.097166,48.213750,187.883416,1739.0,1551.116584,925.573972,93.0
4,1048686,2019-04-30,0xE535FBD1AB5E3C4C21,76967,0,0,0,3+,0xE92ADDF60D9801351C338B2608632B61,2017-10-18,2051-06-01,82800,76967,0.75,no,33,236.097166,48.104375,187.992791,175.0,-12.992791,93.088676,93.0
5,1058489,2019-05-31,0xE535FBD1AB5E3C4C21,74738,0,0,0,3+,0xE92ADDF60D9801351C338B2608632B61,2017-10-18,2050-06-01,82800,74738,0.75,no,32,242.601061,46.711250,195.889811,2229.0,2033.110189,1137.884603,90.0
6,1058230,2019-06-28,0xE535FBD1AB5E3C4C21,74561,0,0,0,3+,0xE92ADDF60D9801351C338B2608632B61,2017-10-18,2050-06-01,82800,74561,0.75,no,32,242.601061,46.600625,196.000436,177.0,-19.000436,90.305921,90.0
7,1161969,2019-07-31,0xE535FBD1AB5E3C4C21,72942,0,0,0,3+,0xE92ADDF60D9801351C338B2608632B61,2017-10-18,2049-10-01,82800,72942,0.75,no,31,249.526636,45.588750,203.937886,1619.0,1415.062114,793.869168,88.0
8,1048687,2019-08-30,0xE535FBD1AB5E3C4C21,72815,0,0,0,3+,0xE92ADDF60D9801351C338B2608632B61,2017-10-18,2049-10-01,82800,72764,0.75,no,31,249.526636,45.477500,204.049136,178.0,-26.049136,87.233891,88.0
9,1091535,2019-09-30,0xE535FBD1AB5E3C4C21,72586,0,0,0,3+,0xE92ADDF60D9801351C338B2608632B61,2017-10-18,2049-10-01,82800,72586,0.75,no,31,249.526636,45.366250,204.160386,178.0,-26.160386,87.186355,88.0


In [103]:
balances_grouped = balances_updated.groupby(["customer_id", 
                                             "account_id", 
                                             "open_date"]).agg({
    
    "original_principal": ["min", "max", "std",  max_minus_min],
    "reporting_date": "count",
    "total_cust_exposure": ["min", "max"],
    "overdraft": ["min", "max", "size", count0, count1, count2plus],
    "consumer_loan": ["min", "max", "size", count0, count1, count2plus],
    "credit_card": ["min", "max", "size", count0, count1, count2plus],
    "customer_rating": ["min", "max", modeX],
    "maturity_date": ["min", "max", max_minus_min],
    "prepayment_status": ["unique"], # TODO - Think about how to spread the target variable... Could be with DUMMIES
    "l_period": ["min", "max", max_minus_min],
    "interest_rate": ["min", "max", max_minus_min, modeX],
    "exp_monthly_payments": ["min", "max", "std",  max_minus_min],
    "current_principal": ["min", "max", max_minus_min],
    "interest_payment": ["min", "max", max_minus_min],
    "principal_payment": ["min", "max", max_minus_min],
    
    "deducted_principal": ["min", "max", max_minus_min], # MAYBE ADD Standard Deviation (to check if the client has tried to reduce the amount of the principal instalment each month)
    "deducted_vs_principal_payment": ["min", "max", "mean", max_minus_min],
    "principal_left_pctg": ["min", "max", max_minus_min]
    
    
})


C:\Users\Mitko\AppData\Local\Temp\ipykernel_5556\965494058.py:14: RuntimeWarning: invalid value encountered in double_scalars
  return x.max() - x.min()


In [104]:
balances_grouped

original_principal  \
                                                                                  min   
customer_id          account_id                         open_date                       
0x000100F424893C746F 0xF7CDAD654B6CE06334F9FE655212AAD1 2017-06-29              64800   
0x000224F02B4B39DDFE 0x5568E6E4AEB2DF832D6444950125EECB 2016-08-16              74880   
0x00025DA4C145ED8EBF 0x8C794E5D7336C4FE59B85E4400973723 2022-04-13              54000   
0x0003C2466FAE7F0BEA 0xAAD9093643AACB7C73D1DAB125D551FE 2018-05-08              93600   
0x0004EAC7528D4869C1 0x1DE89AAB80B0FE12FBA0E35F21DDA118 2021-07-07             118320   
...                                                                               ...   
0xFFF87FAE7F0E3943C2 0x65B3840D27C64133B6B50A188D85B839 2012-03-14              58675   
0xFFF9156414DABBF1B7 0xC1F24B7F9836656402D5E230EB2850F5 2017-08-04              90780   
0xFFF97F0223324E0A41 0x82CB3ECA8054330F90873C4C500D7E4B 2020-10-15             195600   
0xFFFAF7DD151CBA684C 0x89366BD6313790B2F05DA41D61D2685E 2020-01-15              80172   
0xFFFFE568ED457ED855 0xBEACB8DEA2E50735BCA50D298D1A55C3 2022-06-02             591942   

                                                                            \
                                                                       max   
customer_id          account_id                         open_date            
0x000100F424893C746F 0xF7CDAD654B6CE06334F9FE655212AAD1 2017-06-29   64800   
0x000224F02B4B39DDFE 0x5568E6E4AEB2DF832D6444950125EECB 2016-08-16   74880   
0x00025DA4C145ED8EBF 0x8C794E5D7336C4FE59B85E4400973723 2022-04-13   54000   
0x0003C2466FAE7F0BEA 0xAAD9093643AACB7C73D1DAB125D551FE 2018-05-08   93600   
0x0004EAC7528D4869C1 0x1DE89AAB80B0FE12FBA0E35F21DDA118 2021-07-07  118320   
...                                                                    ...   
0xFFF87FAE7F0E3943C2 0x65B3840D27C64133B6B50A188D85B839 2012-03-14   58675   
0xFFF9156414DABBF1B7 0xC1F24B7F9836656402D5E230EB2850F5 2017-08-04   90780   
0xFFF97F0223324E0A41 0x82CB3ECA8054330F90873C4C500D7E4B 2020-10-15  195600   
0xFFFAF7DD151CBA684C 0x89366BD6313790B2F05DA41D61D2685E 2020-01-15   80172   
0xFFFFE568ED457ED855 0xBEACB8DEA2E50735BCA50D298D1A55C3 2022-06-02  591942   

                                                                         \
                                                                    std   
customer_id          account_id                         open_date         
0x000100F424893C746F 0xF7CDAD654B6CE06334F9FE655212AAD1 2017-06-29  0.0   
0x000224F02B4B39DDFE 0x5568E6E4AEB2DF832D6444950125EECB 2016-08-16  0.0   
0x00025DA4C145ED8EBF 0x8C794E5D7336C4FE59B85E4400973723 2022-04-13  0.0   
0x0003C2466FAE7F0BEA 0xAAD9093643AACB7C73D1DAB125D551FE 2018-05-08  0.0   
0x0004EAC7528D4869C1 0x1DE89AAB80B0FE12FBA0E35F21DDA118 2021-07-07  0.0   
...                                                                 ...   
0xFFF87FAE7F0E3943C2 0x65B3840D27C64133B6B50A188D85B839 2012-03-14  0.0   
0xFFF9156414DABBF1B7 0xC1F24B7F9836656402D5E230EB2850F5 2017-08-04  0.0   
0xFFF97F0223324E0A41 0x82CB3ECA8054330F90873C4C500D7E4B 2020-10-15  0.0   
0xFFFAF7DD151CBA684C 0x89366BD6313790B2F05DA41D61D2685E 2020-01-15  0.0   
0xFFFFE568ED457ED855 0xBEACB8DEA2E50735BCA50D298D1A55C3 2022-06-02  NaN   

                                                                                  \
                                                                   max_minus_min   
customer_id          account_id                         open_date                  
0x000100F424893C746F 0xF7CDAD654B6CE06334F9FE655212AAD1 2017-06-29             0   
0x000224F02B4B39DDFE 0x5568E6E4AEB2DF832D6444950125EECB 2016-08-16             0   
0x00025DA4C145ED8EBF 0x8C794E5D7336C4FE59B85E4400973723 2022-04-13             0   
0x0003C2466FAE7F0BEA 0xAAD9093643AACB7C73D1DAB125D551FE 2018-05-08             0   
0x0004EAC7528D4869C1 0x1DE89AAB80B0FE12FBA0E35F21DDA118 2021-07-07             0   
... 

### Explore grouped data

In [105]:
# for col in balances_grouped.columns:
#     print(f"{col}: {balances_grouped[col].value_counts()[:2]}")

In [106]:
balances_grouped_res_idx = balances_grouped.reset_index()

In [107]:
balances_grouped_res_idx.columns = balances_grouped_res_idx.columns.get_level_values(0) + '_' +  balances_grouped_res_idx.columns.get_level_values(1)

In [108]:
# TO EXCECUTE AFTER MERGING WITH OTHER CSV's
def make_unique_ids(size):
    
    
    new_id = "".join(random.choice(string.ascii_letters) for _ in range(size))
    pass

In [109]:
val_counts = balances_grouped_res_idx["customer_id_"].value_counts()
complicated_clients = list(val_counts[val_counts>1].index)

In [110]:
balances_grouped_res_idx.shape

(39704, 65)

In [111]:
balances_prep_simple_v1 = balances_grouped_res_idx[~balances_grouped_res_idx['customer_id_'].isin(complicated_clients)]
balances_prep_simple_v1.shape

(39339, 65)

In [113]:
balances_prep_simple_v1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 39339 entries, 0 to 39703
Data columns (total 65 columns):
 #   Column                                       Non-Null Count  Dtype          
---  ------                                       --------------  -----          
 0   customer_id_                                 39339 non-null  object         
 1   account_id_                                  39339 non-null  object         
 2   open_date_                                   39339 non-null  datetime64[ns] 
 3   original_principal_min                       39339 non-null  int64          
 4   original_principal_max                       39339 non-null  int64          
 5   original_principal_std                       38730 non-null  float64        
 6   original_principal_max_minus_min             39339 non-null  int64          
 7   reporting_date_count                         39339 non-null  int64          
 8   total_cust_exposure_min                      39339 non-null  int64

In [118]:
balances_prep_simple_v1["maturity_date_max_minus_min"] = balances_prep_simple_v1["maturity_date_max_minus_min"].apply(lambda x: x.days)

C:\Users\Mitko\AppData\Local\Temp\ipykernel_5556\3078281136.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  balances_prep_simple_v1["maturity_date_max_minus_min"] = balances_prep_simple_v1["maturity_date_max_minus_min"].apply(lambda x: x.days)


In [123]:
balances_prep_simple_v1.fillna(0, inplace=True)

C:\Users\Mitko\AppData\Local\Temp\ipykernel_5556\3561159343.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  balances_prep_simple_v1.fillna(0, inplace=True)


In [143]:
ds = balances_prep_simple_v1.isin([np.inf, -np.inf, "inf"])
print(ds)


       customer_id_  account_id_  open_date_  original_principal_min  \
0             False        False       False                   False   
1             False        False       False                   False   
2             False        False       False                   False   
3             False        False       False                   False   
4             False        False       False                   False   
...             ...          ...         ...                     ...   
39699         False        False       False                   False   
39700         False        False       False                   False   
39701         False        False       False                   False   
39702         False        False       False                   False   
39703         False        False       False                   False   

       original_principal_max  original_principal_std  \
0                       False                   False   
1                    

In [144]:
ds.sum()

customer_id_                                   0
account_id_                                    0
open_date_                                     0
original_principal_min                         0
original_principal_max                         0
original_principal_std                         0
original_principal_max_minus_min               0
reporting_date_count                           0
total_cust_exposure_min                        0
total_cust_exposure_max                        0
overdraft_min                                  0
overdraft_max                                  0
overdraft_size                                 0
overdraft_count0                               0
overdraft_count1                               0
overdraft_count2plus                           0
consumer_loan_min                              0
consumer_loan_max                              0
consumer_loan_size                             0
consumer_loan_count0                           0
consumer_loan_count1

In [ ]:
X_train_fs

In [152]:
inf_cols = ["exp_monthly_payments_min", 
"exp_monthly_payments_max", 
"exp_monthly_payments_max_minus_min", 
"principal_payment_min", "principal_payment_max", 
"principal_payment_max_minus_min", 
"deducted_vs_principal_payment_min", 
"deducted_vs_principal_payment_max", 
"deducted_vs_principal_payment_max_minus_min"]

In [153]:
for col in inf_cols:
    balances_prep_simple_v1["exp_monthly_payments_min"].replace([np.inf, -np.inf], 0, inplace=True)

C:\Users\Mitko\AppData\Local\Temp\ipykernel_5556\1046925887.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  balances_prep_simple_v1["exp_monthly_payments_min"].replace([np.inf, -np.inf], 0, inplace=True)


In [155]:
balances_prep_simple_v1.fillna(0, inplace=True)

C:\Users\Mitko\AppData\Local\Temp\ipykernel_5556\3561159343.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  balances_prep_simple_v1.fillna(0, inplace=True)


In [157]:
def new_rating(row, col_name):
    
    val = row
    if "+" in val:
        res_val = float(val[0]+".5")
    elif "-" in val:
        res = float(val[0]+".9")
        res_val = res - 1
    else:
        res_val = float(val)
    
    return res_val

In [158]:
rating_cols = ["customer_rating_min", "customer_rating_max", "customer_rating_modeX"]
for col in rating_cols:
#     print(col)
    balances_prep_simple_v1[col] = balances_prep_simple_v1[col].apply(lambda x: new_rating(x, col_name=col))

C:\Users\Mitko\AppData\Local\Temp\ipykernel_5556\17875453.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  balances_prep_simple_v1[col] = balances_prep_simple_v1[col].apply(lambda x: new_rating(x, col_name=col))


In [159]:
balances_prep_simple_v1.to_csv("balances_prep_simple_v2.csv")

In [56]:
balances_grouped_res_idx.loc[balances_grouped_res_idx["customer_id_"] == "0x36E2A445DD2A871643"]   

,customer_id_,account_id_,open_date_,original_principal_min,original_principal_max,original_principal_std,original_principal_max_minus_min,reporting_date_count,total_cust_exposure_min,total_cust_exposure_max,overdraft_min,overdraft_max,overdraft_size,overdraft_count0,overdraft_count1,overdraft_count2plus,consumer_loan_min,consumer_loan_max,consumer_loan_size,consumer_loan_count0,consumer_loan_count1,consumer_loan_count2plus,credit_card_min,credit_card_max,credit_card_size,credit_card_count0,credit_card_count1,credit_card_count2plus,customer_rating_min,customer_rating_max,customer_rating_modeX,maturity_date_min,maturity_date_max,maturity_date_max_minus_min,prepayment_status_unique,l_period_min,l_period_max,l_period_max_minus_min,interest_rate_min,interest_rate_max,interest_rate_max_minus_min,interest_rate_modeX,exp_monthly_payments_min,exp_monthly_payments_max,exp_monthly_payments_std,exp_monthly_payments_max_minus_min,current_principal_min,current_principal_max,current_principal_max_minus_min,interest_payment_min,interest_payment_max,interest_payment_max_minus_min,principal_payment_min,principal_payment_max,principal_payment_max_minus_min,deducted_principal_min,deducted_principal_max,deducted_principal_max_minus_min,deducted_vs_principal_payment_min,deducted_vs_principal_payment_max,deducted_vs_principal_payment_mean,deducted_vs_principal_payment_max_minus_min,principal_left_pctg_min,principal_left_pctg_max,principal_left_pctg_max_minus_min
8459,0x36E2A445DD2A871643,0x6DBAEC05D2B8051130BC9FFC5B041666,2005-06-13,61022,61022,0.0,0,8,20866,22964,0,0,8,8,0,0,0,0,8,8,0,0,0,0,8,8,0,0,7,7+,7,2024-06-15,2024-06-15,0 days,[no],19,19,0,5.895,5.922,0.027,5.918,445.529836,446.465052,0.29792,0.935217,20866,22964,2098,102.504225,113.250793,10.746568,333.075643,343.025611,9.949967,295.0,38058.0,37763.0,-39.593537,37724.924357,4681.335635,37764.517894,34.0,38.0,4.0
8460,0x36E2A445DD2A871643,0x6DBAEC05D2B8051130BC9FFC5B041666,2005-06-29,61022,61022,0.0,0,36,61636,86542,0,0,36,36,0,0,0,0,36,36,0,0,0,0,36,36,0,0,4,8+,5+,2024-06-15,2024-06-15,0 days,[no],18,18,0,5.701,5.894,0.193,5.856,452.443518,459.015065,1.90943,6.571547,9135,20560,11425,43.703362,100.983867,57.280504,358.031198,410.097968,52.066770,308.0,45664.0,45356.0,-50.887422,45303.594898,34293.960183,45354.482320,15.0,34.0,19.0
8461,0x36E2A445DD2A871643,0x6DBAEC05D2B8051130BC9FFC5B041666,2019-08-07,66480,66480,0.0,0,36,61636,86542,0,0,36,36,0,0,0,0,36,36,0,0,0,0,36,36,0,0,4,8+,5+,2031-08-07,2031-08-07,0 days,[no],12,12,0,3.000,3.000,0.000,3.0,550.312591,550.312591,0.00000,0.000000,52501,65982,13481,131.252500,164.955000,33.702500,385.357591,419.060091,33.702500,-45353.0,414.0,45767.0,-45739.300091,-3.170091,-35958.076758,45736.130000,79.0,99.0,20.0


In [57]:
balances_updated.loc[balances_updated["customer_id"] == "0x36E2A445DD2A871643"]

,index,reporting_date,customer_id,total_cust_exposure,overdraft,consumer_loan,credit_card,customer_rating,account_id,open_date,maturity_date,original_principal,current_principal,interest_rate,prepayment_status,l_period,exp_monthly_payments,interest_payment,principal_payment,deducted_principal,deducted_vs_principal_payment,deducted_vs_principal_payment_ratio,principal_left_pctg
0,1218426,2018-12-31,0x36E2A445DD2A871643,22964,0,0,0,7,0x6DBAEC05D2B8051130BC9FFC5B041666,2005-06-13,2024-06-15,61022,22964,5.918,no,19,446.326437,113.250793,333.075643,38058.0,37724.924357,11426.233272,38.0
1,834511,2019-01-31,0x36E2A445DD2A871643,22669,0,0,0,7,0x6DBAEC05D2B8051130BC9FFC5B041666,2005-06-13,2024-06-15,61022,22669,5.922,no,19,446.465052,111.871515,334.593537,295.0,-39.593537,88.166676,37.0
2,1066326,2019-02-28,0x36E2A445DD2A871643,22372,0,0,0,7,0x6DBAEC05D2B8051130BC9FFC5B041666,2005-06-13,2024-06-15,61022,22372,5.918,no,19,446.326437,110.331247,335.995190,297.0,-38.995190,88.394123,37.0
3,875738,2019-03-29,0x36E2A445DD2A871643,22074,0,0,0,7,0x6DBAEC05D2B8051130BC9FFC5B041666,2005-06-13,2024-06-15,61022,22074,5.918,no,19,446.326437,108.861610,337.464827,298.0,-39.464827,88.305499,36.0
4,1147901,2019-04-30,0x36E2A445DD2A871643,21774,0,0,0,7+,0x6DBAEC05D2B8051130BC9FFC5B041666,2005-06-13,2024-06-15,61022,21774,5.918,no,19,446.326437,107.382110,338.944327,300.0,-38.944327,88.510111,36.0
5,907477,2019-05-31,0x36E2A445DD2A871643,21473,0,0,0,7,0x6DBAEC05D2B8051130BC9FFC5B041666,2005-06-13,2024-06-15,61022,21473,5.918,no,19,446.326437,105.897678,340.428758,301.0,-39.428758,88.417912,35.0
6,911599,2019-06-28,0x36E2A445DD2A871643,21170,0,0,0,7,0x6DBAEC05D2B8051130BC9FFC5B041666,2005-06-13,2024-06-15,61022,21170,5.910,no,19,446.049273,104.262250,341.787023,303.0,-38.787023,88.651698,35.0
7,1147599,2019-07-31,0x36E2A445DD2A871643,20866,0,0,0,7,0x6DBAEC05D2B8051130BC9FFC5B041666,2005-06-13,2024-06-15,61022,20866,5.895,no,19,445.529836,102.504225,343.025611,304.0,-39.025611,88.623120,34.0
8,1063655,2019-08-30,0x36E2A445DD2A871643,86542,0,0,0,7+,0x6DBAEC05D2B8051130BC9FFC5B041666,2019-08-07,2031-08-07,66480,65982,3.000,no,12,550.312591,164.955000,385.357591,-45116.0,-45501.357591,-11707.567474,99.0
9,1063637,2019-08-30,0x36E2A445DD2A871643,86542,0,0,0,7+,0x6DBAEC05D2B8051130BC9FFC5B041666,2005-06-29,2024-06-15,61022,20560,5.894,no,18,459.015065,100.983867,358.031198,45422.0,45063.968802,12686.603912,34.0


In [160]:
balances_grouped_res_idx.head()

,customer_id_,account_id_,open_date_,original_principal_min,original_principal_max,original_principal_std,original_principal_max_minus_min,reporting_date_count,total_cust_exposure_min,total_cust_exposure_max,overdraft_min,overdraft_max,overdraft_size,overdraft_count0,overdraft_count1,overdraft_count2plus,consumer_loan_min,consumer_loan_max,consumer_loan_size,consumer_loan_count0,consumer_loan_count1,consumer_loan_count2plus,credit_card_min,credit_card_max,credit_card_size,credit_card_count0,credit_card_count1,credit_card_count2plus,customer_rating_min,customer_rating_max,customer_rating_modeX,maturity_date_min,maturity_date_max,maturity_date_max_minus_min,prepayment_status_unique,l_period_min,l_period_max,l_period_max_minus_min,interest_rate_min,interest_rate_max,interest_rate_max_minus_min,interest_rate_modeX,exp_monthly_payments_min,exp_monthly_payments_max,exp_monthly_payments_std,exp_monthly_payments_max_minus_min,current_principal_min,current_principal_max,current_principal_max_minus_min,interest_payment_min,interest_payment_max,interest_payment_max_minus_min,principal_payment_min,principal_payment_max,principal_payment_max_minus_min,deducted_principal_min,deducted_principal_max,deducted_principal_max_minus_min,deducted_vs_principal_payment_min,deducted_vs_principal_payment_max,deducted_vs_principal_payment_mean,deducted_vs_principal_payment_max_minus_min,principal_left_pctg_min,principal_left_pctg_max,principal_left_pctg_max_minus_min
0,0x000100F424893C746F,0xF7CDAD654B6CE06334F9FE655212AAD1,2017-06-29,64800,64800,0.0,0,44,56155,63355,0,0,44,44,0,0,0,0,44,44,0,0,0,0,44,44,0,0,3,5+,3+,2042-06-29,2042-06-29,0 days,[no],24,24,0,3.70,3.70,0.0,3.70,339.819283,339.819283,0.0,0.0,56155,62342,6187,173.144583,192.221167,19.076583,147.598116,166.674700,19.076583,0.0,2458.0,2458.0,-164.251200,2310.401884,39.390899,2474.653083,87.0,96.0,9.0
1,0x000224F02B4B39DDFE,0x5568E6E4AEB2DF832D6444950125EECB,2016-08-16,74880,74880,0.0,0,11,70449,72710,0,0,11,11,0,0,0,0,11,11,0,0,0,0,11,11,0,0,4,5+,4,2042-08-16,2042-08-16,0 days,[refinanced other banks],25,25,0,3.90,3.90,0.0,3.90,391.121433,391.121433,0.0,0.0,69729,71273,1544,226.619250,231.637250,5.018000,159.484183,164.502183,5.018000,147.0,3607.0,3460.0,-12.961933,3447.515817,306.287522,3460.477750,93.0,95.0,2.0
2,0x00025DA4C145ED8EBF,0x8C794E5D7336C4FE59B85E4400973723,2022-04-13,54000,54000,0.0,0,3,53248,54000,0,0,3,3,0,0,0,0,3,3,0,0,0,0,3,3,0,0,5+,5+,5+,2033-04-13,2033-04-13,0 days,[no],11,11,0,2.48,2.48,0.0,2.48,467.844781,467.844781,0.0,0.0,53248,54000,752,110.045867,111.600000,1.554133,356.244781,357.798915,1.554133,0.0,398.0,398.0,-356.244781,40.932685,-106.370337,397.177467,99.0,100.0,1.0
3,0x0003C2466FAE7F0BEA,0xAAD9093643AACB7C73D1DAB125D551FE,2018-05-08,93600,93600,0.0,0,44,87324,92598,0,0,44,44,0,0,0,0,44,44,0,0,0,0,44,44,0,0,3,6+,3+,2053-05-08,2053-05-08,0 days,[no],35,35,0,3.35,3.35,0.0,3.35,378.749527,378.749527,0.0,0.0,87324,92598,5274,243.779500,258.502750,14.723250,120.246777,134.970027,14.723250,0.0,1002.0,1002.0,-133.096819,881.753223,15.045854,1014.850042,93.0,99.0,6.0
4,0x0004EAC7528D4869C1,0x1DE89AAB80B0FE12FBA0E35F21DDA118,2021-07-07,118320,118320,0.0,0,12,95275,116897,0,0,12,12,0,0,0,0,12,12,0,0,0,0,12,12,0,0,3,4-,4-,2051-07-07,2051-07-07,0 days,[no],29,29,0,2.49,2.49,0.0,2.49,477.741699,477.741699,0.0,0.0,95275,116897,21622,197.695625,242.561275,44.865650,235.180424,280.046074,44.865650,-21622.0,22149.0,43771.0,-21857.180424,21870.813126,-65.958891,43727.993550,81.0,99.0,18.0


In [ ]:
len(balances_grouped_res_idx.loc[balances_grouped_res_idx["customer_id_"] == "0x53F89E119E11038EDB"]   )

In [ ]:
balances_grouped_res_idx.to_csv("balances_grouped_res_idx_v3.csv")

### Create target values

1. Reformat the target to be a Binary Prepaid(no, yes)
2. After modeling - create targets for 3 months, 6 months, 12 months

In [ ]:
balances_grouped.head()

In [ ]:
balances_grouped["prepayment_status"].value_counts()

## Other code

In [ ]:
balances_grouped

In [ ]:
balances_grouped.index

In [ ]:
balances_grouped.columns = balances_grouped.columns.get_level_values(0) + '_' +  balances_grouped.columns.get_level_values(1)

In [ ]:
balances_grouped.reset_index(inplace=True)

In [ ]:
balances_grouped

In [ ]:
balances_grouped

In [ ]:
balances_grouped.columns